# Save signal maps

In [1]:
import os
import numpy as np
import pandas as pd
import rasterio as rio
import matplotlib.pyplot as plt
import xarray as xr
import datetime as dt
import rioxarray
import seaborn as sns
import geopandas as gpd
from glob import glob
import math

In [2]:
# function to rewrite coordinates from metadata
def coord_range(ds):
    latrange = np.linspace(float(ds.attrs['Y_FIRST']),
                           ((float(ds.attrs['Y_STEP'])*float(ds.attrs['LENGTH']))+float(ds.attrs['Y_FIRST'])),
                           int(ds.attrs['LENGTH']))
    lonrange = np.linspace(float(ds.attrs['X_FIRST']),
                           ((float(ds.attrs['X_STEP'])*float(ds.attrs['WIDTH']))+float(ds.attrs['X_FIRST'])),
                           int(ds.attrs['WIDTH']))
    return latrange, lonrange

def mintpy2d_to_xarray(fn, crs):
    ds = xr.open_dataset(fn)
    ds = ds.rename_dims({'phony_dim_0':'y',
                         'phony_dim_1':'x'
                        })
    latrange, lonrange = coord_range(ds)
    ds = ds.assign_coords({'y': ('y', latrange),
                           'x': ('x', lonrange)})
    ds = ds.rio.write_crs(crs)
    
    return ds

In [3]:
# add additional rasters for analysis
def rasters_to_ds(ds, granule_path):
    os.chdir(granule_path)
    
    slope_ds = xr.open_dataset(glob('*slope.tif')[0])
    aspect_ds = xr.open_dataset(glob('*aspect.tif')[0])
    heading_ds = xr.open_dataset(glob('*lv_phi.tif')[0])
    incidence_ds = xr.open_dataset(glob('*inc_map_ell.tif')[0])
    
    slope_ds = slope_ds.rio.reproject_match(ds.velocity)
    aspect_ds = aspect_ds.rio.reproject_match(ds.velocity)
    heading_ds = heading_ds.rio.reproject_match(ds.velocity)
    incidence_ds = incidence_ds.rio.reproject_match(ds.velocity)

    # add coherence and dem to unw_phase dataset
    ds['slope'] = (('band', 'y', 'x'), slope_ds.band_data.values)
    ds['aspect'] = (('band', 'y', 'x'), aspect_ds.band_data.values)
    ds['heading'] = (('band', 'y', 'x'), heading_ds.band_data.values)
    ds['incidence'] = (('band', 'y', 'x'), incidence_ds.band_data.values)
    
    # remove band coordinate
    ds = ds.squeeze()
    
    return ds
                  
def hyp3_to_downslope(ds):
    ds['slope'] = np.radians(ds.slope)
    ds['aspect'] = np.radians(ds.aspect)
    a = ds.heading - ds.aspect

    ds['downslope_velocity'] = ds.velocity/(np.sin(a)*np.sin(ds.incidence)*np.cos(ds.slope)+np.cos(ds.incidence)*np.sin(ds.slope))
    
    return ds

In [4]:
orbit_list = ['AT137']
year_list = ['2017', '2018', '2019', '2020', '2021']

In [5]:
# function to clip files and run mintpy for multiple years 
def save_veloc_multiyear(orbit_list, year_list, crs=32612):
    # hardcoded paths for now 
    home_path = '/mnt/d/indennt'
    for orbit in orbit_list:
        for year in year_list:
            print(f'working on {orbit}, {year}')
            mintpy_path = f'{home_path}/mintpy_app/{orbit}/mintpy_{year}_uncorrected'

            ds = mintpy2d_to_xarray(f'{mintpy_path}/velocity.h5', crs)
        
            # save velocity
            vel_ds = ds.velocity.rio.write_nodata(0, inplace=True)
            vel_ds.rio.to_raster(f'{home_path}/signal_maps_app/{orbit}/{year}_signal_uncorrected.tif')
            
            # #downslope velocity
            # raster_path = glob(f'{home_path}/hyp3_app/{orbit}/{year}/*')[0]
            # ds = rasters_to_ds(ds, raster_path)
            # ds = hyp3_to_downslope(ds)
            # downslope_ds = ds.downslope_velocity.rio.write_nodata(0, inplace=True)
            # downslope_ds.rio.to_raster(f'{home_path}/signal_maps_app/{orbit}/{year}_downslope_signal.tif')
        
            # # coherence 
            # corr_ds = mintpy2d_to_xarray(f'{mintpy_path}/avgSpatialCoh.h5', crs)
            # corr_ds = corr_ds.rio.reproject_match(vel_ds)
            # corr_ds = corr_ds.coherence.rio.write_nodata(0, inplace=True)
            # corr_ds.rio.to_raster(f'{home_path}/signal_maps_app/{orbit}/{year}_corr.tif')

In [6]:
save_veloc_multiyear(orbit_list, year_list)

working on AT137, 2017
working on AT137, 2018
working on AT137, 2019
working on AT137, 2020
working on AT137, 2021
